In [8]:
!pip install keras --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.


In [2]:
!pip install keras_nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 73.1 MB/s eta 0:00:00


In [1]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"  # or "tensorflow" or "torch"

import keras_nlp
import keras

# Use mixed precision to speed up all training in this guide.
keras.mixed_precision.set_global_policy("mixed_float16")


In [2]:
print(keras.__version__)
print(keras_nlp.__version__)

3.0.5
0.8.2


In [5]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!# Remove unsupervised examples
!rm -r aclImdb/train/unsup



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  4059k      0  0:00:20  0:00:20 --:--:-- 3976k


In [3]:
BATCH_SIZE = 16
imdb_train = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=BATCH_SIZE,
)
imdb_test = keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=BATCH_SIZE,
)

# Inspect first review
# Format is (review text tensor, label tensor)
print(imdb_train.unbatch().take(1).get_single_element())


Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
(<tf.Tensor: shape=(), dtype=string, numpy=b"I'm sorry but I didn't like this doc very much. I can think of a million ways it could have been better. The people who made it obviously don't have much imagination. The interviews aren't very interesting and no real insight is offered. The footage isn't assembled in a very informative way, either. It's too bad because this is a movie that really deserves spellbinding special features. One thing I'll say is that Isabella Rosselini gets more beautiful the older she gets. All considered, this only gets a '4.'">, <tf.Tensor: shape=(), dtype=int32, numpy=0>)


In [4]:
classifier = keras_nlp.models.BertClassifier.from_preset("bert_tiny_en_uncased_sst2")
# Note: batched inputs expected so must wrap string in iterable
classifier.predict(["I love modular workflows in keras-nlp!"])


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'loss_scale_optimizer', because it has 4 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 0 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


array([[-1.539,  1.543]], dtype=float16)

In [5]:
classifier.evaluate(imdb_test)

1563/1563 ━━━━━━━━━━━━━━━━━━━━ 14s 7ms/step - loss: 0.4676 - sparse_categorical_accuracy: 0.7832


[0.4629317820072174, 0.783519983291626]

### Fine tuning a pretrained BERT backbone

In [6]:
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=2,
)
classifier.fit(
    imdb_train,
    validation_data=imdb_test,
    epochs=2,
)


100%|██████████| 507/507 [00:00<00:00, 316kB/s]
100%|██████████| 16.8M/16.8M [00:02<00:00, 6.96MB/s]
100%|██████████| 547/547 [00:00<00:00, 479kB/s]
100%|██████████| 226k/226k [00:00<00:00, 306kB/s]


Epoch 1/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 94s 43ms/step - loss: 0.5178 - sparse_categorical_accuracy: 0.7299 - val_loss: 0.3017 - val_sparse_categorical_accuracy: 0.8726
Epoch 2/2
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 37s 24ms/step - loss: 0.2873 - sparse_categorical_accuracy: 0.8804 - val_loss: 0.2867 - val_sparse_categorical_accuracy: 0.8811


### Fine tuning with user-controlled preprocessing

In [7]:
import tensorflow as tf

preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_tiny_en_uncased",
    sequence_length=512,
)

# Apply the preprocessor to every sample of train and test data using `map()`.
# [`tf.data.AUTOTUNE`](https://www.tensorflow.org/api_docs/python/tf/data/AUTOTUNE) and `prefetch()` are options to tune performance, see
# https://www.tensorflow.org/guide/data_performance for details.

# Note: only call `cache()` if you training data fits in CPU memory!
imdb_train_cached = (
    imdb_train.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)
imdb_test_cached = (
    imdb_test.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)

classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased", preprocessor=None, num_classes=2
)
classifier.fit(
    imdb_train_cached,
    validation_data=imdb_test_cached,
    epochs=3,
)


Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 80s 35ms/step - loss: 0.5180 - sparse_categorical_accuracy: 0.7291 - val_loss: 0.3150 - val_sparse_categorical_accuracy: 0.8660
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 76s 49ms/step - loss: 0.2893 - sparse_categorical_accuracy: 0.8825 - val_loss: 0.3199 - val_sparse_categorical_accuracy: 0.8689
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 72s 46ms/step - loss: 0.2201 - sparse_categorical_accuracy: 0.9143 - val_loss: 0.3073 - val_sparse_categorical_accuracy: 0.8792


In [8]:
tokenizer = keras_nlp.models.BertTokenizer.from_preset("bert_tiny_en_uncased")
tokenizer(["I love modular workflows!", "Libraries over frameworks!"])

# Write your own packer or use one of our `Layers`
packer = keras_nlp.layers.MultiSegmentPacker(
    start_value=tokenizer.cls_token_id,
    end_value=tokenizer.sep_token_id,
    # Note: This cannot be longer than the preset's `sequence_length`, and there
    # is no check for a custom preprocessor!
    sequence_length=64,
)


# This function that takes a text sample `x` and its
# corresponding label `y` as input and converts the
# text into a format suitable for input into a BERT model.
def preprocessor(x, y):
    token_ids, segment_ids = packer(tokenizer(x))
    x = {
        "token_ids": token_ids,
        "segment_ids": segment_ids,
        "padding_mask": token_ids != 0,
    }
    return x, y


imdb_train_preprocessed = imdb_train.map(preprocessor, tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)
imdb_test_preprocessed = imdb_test.map(preprocessor, tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)

# Preprocessed example
print(imdb_train_preprocessed.unbatch().take(1).get_single_element())


({'token_ids': <tf.Tensor: shape=(64,), dtype=int32, numpy=
array([  101,  1999,  2023, 13939,  1011,  3768,  8447, 17417,  9289,
       16976,  1010,  1996,  2088, 17567,  2013,  1037,  7037, 11629,
        1997, 14106,  1998,  6144,  1010,  2021,  2053,  2028,  3849,
        2000,  2022,  2205,  5191,  2055,  2009,  1999,  2023,  3694,
        1011,  1062,  2143,  2029,  6436,  2015,  2625, 14062,  2000,
       13243,  3393,  5470,  2226,  1005,  1055,  3297,  2460,  3117,
        2084,  2026,  2388,  1005,  1055,  5869,  8490,  2532, 17974,
         102], dtype=int32)>, 'segment_ids': <tf.Tensor: shape=(64,), dtype=int32, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int32)>, 'padding_mask': <tf.Tensor: shape=(64,), dtype=bool, numpy=
array([ True,  True,  True,  True,  True,  True,  True,  True,  T

### Fine tuning with a custom model

In [9]:
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_tiny_en_uncased")
backbone = keras_nlp.models.BertBackbone.from_preset("bert_tiny_en_uncased")

imdb_train_preprocessed = (
    imdb_train.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)
imdb_test_preprocessed = (
    imdb_test.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)

backbone.trainable = False
inputs = backbone.input
sequence = backbone(inputs)["sequence_output"]
for _ in range(2):
    sequence = keras_nlp.layers.TransformerEncoder(
        num_heads=2,
        intermediate_dim=512,
        dropout=0.1,
    )(sequence)
# Use [CLS] token output to classify
outputs = keras.layers.Dense(2)(sequence[:, backbone.cls_token_index, :])

model = keras.Model(inputs, outputs)
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.AdamW(5e-5),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    jit_compile=True,
)
model.summary()
model.fit(
    imdb_train_preprocessed,
    validation_data=imdb_test_preprocessed,
    epochs=3,
)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer) │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ segment_ids (InputLayer)  │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ token_ids (InputLayer)    │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_backbone             │ [(None, 128), (None,   │      4,385,920 │ padding_mask[0][0],    │
│ (BertBackbone)            │ None, 128)]            │                │ segment_ids[0][0],     │
│                           │                        │                │ token_ids[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, None, 128)      │        198,272 │ bert_backbone[0][1]    │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder_1     │ (None, None, 128)      │        198,272 │ transformer_encoder[0… │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ get_item_4 (GetItem)      │ (None, 128)            │              0 │ transformer_encoder_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 2)              │            258 │ get_item_4[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,782,722 (18.24 MB)

 Trainable params: 396,802 (1.51 MB)

 Non-trainable params: 4,385,920 (16.73 MB)

Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 118s 52ms/step - loss: 0.6325 - sparse_categorical_accuracy: 0.6497 - val_loss: 0.5104 - val_sparse_categorical_accuracy: 0.7512
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 69s 23ms/step - loss: 0.5045 - sparse_categorical_accuracy: 0.7540 - val_loss: 0.4234 - val_sparse_categorical_accuracy: 0.8040
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 36s 23ms/step - loss: 0.4529 - sparse_categorical_accuracy: 0.7841 - val_loss: 0.4005 - val_sparse_categorical_accuracy: 0.8176


### Pretraining a backbone model

#### Preprocessing

In [11]:
# All BERT `en` models have the same vocabulary, so reuse preprocessor from
# "bert_tiny_en_uncased"
preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_tiny_en_uncased",
    sequence_length=256,
)
packer = preprocessor.packer
tokenizer = preprocessor.tokenizer

# keras.Layer to replace some input tokens with the "[MASK]" token
masker = keras_nlp.layers.MaskedLMMaskGenerator(
    vocabulary_size=tokenizer.vocabulary_size(),
    mask_selection_rate=0.25,
    mask_selection_length=64,
    mask_token_id=tokenizer.token_to_id("[MASK]"),
    unselectable_token_ids=[
        tokenizer.token_to_id(x) for x in ["[CLS]", "[PAD]", "[SEP]"]
    ],
)


def preprocess(inputs, label):
    inputs = preprocessor(inputs)
    masked_inputs = masker(inputs["token_ids"])
    # Split the masking layer outputs into a (features, labels, and weights)
    # tuple that we can use with keras.Model.fit().
    features = {
        "token_ids": masked_inputs["token_ids"],
        "segment_ids": inputs["segment_ids"],
        "padding_mask": inputs["padding_mask"],
        "mask_positions": masked_inputs["mask_positions"],
    }
    labels = masked_inputs["mask_ids"]
    weights = masked_inputs["mask_weights"]
    return features, labels, weights


pretrain_ds = imdb_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).prefetch(
    tf.data.AUTOTUNE
)
pretrain_val_ds = imdb_test.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

# Tokens with ID 103 are "masked"
print(pretrain_ds.unbatch().take(1).get_single_element())


({'token_ids': <tf.Tensor: shape=(256,), dtype=int32, numpy=
array([  101,  1999,  1996,  5365,  2225,   103,  4446,  2398,  2020,
        1037,   103,   103,  2040,  2043,   103,  2234,  2000,  2237,
         103,  2112,  6340,  3492,  9532,  1012,  2044,   103,   103,
        4439,  2001,  1037,  9479,   103,   103,  1999,  1996,  2154,
        2077,   103,  2288,  2000,  2035,  3033,  1997,  1996,  3915,
        1012,  1026,   103,   103,  1028,  1026,  7987,  1013,  1028,
        1996,  5225,  2869,   103,  2499,  2005,  4446,  5795, 13097,
        5684,  2004,  2209,  2011,  4388, 13779,   103,  2053,  2367,
         103,  2021, 13779,  2001,  1037,  2158,  1997,  2035,  2449,
         103,   103,   103,  1037,   103,  2000,   103,  1998,  5086,
        1037,  2327,   103,  2000,  2079,  2009,  1012,  1026,  7987,
        1013,  1028,  1026,  7987,  1013,  1028,  2007,   103,  2146,
        2448,  1997,   103, 26100,  1998,  1996,  2755,  2008,  1996,
       24945,  4370,  2007,  

In [12]:
# BERT backbone
backbone = keras_nlp.models.BertBackbone(
    vocabulary_size=tokenizer.vocabulary_size(),
    num_layers=2,
    num_heads=2,
    hidden_dim=128,
    intermediate_dim=512,
)

# Language modeling head
mlm_head = keras_nlp.layers.MaskedLMHead(
    token_embedding=backbone.token_embedding,
)

inputs = {
    "token_ids": keras.Input(shape=(None,), dtype=tf.int32, name="token_ids"),
    "segment_ids": keras.Input(shape=(None,), dtype=tf.int32, name="segment_ids"),
    "padding_mask": keras.Input(shape=(None,), dtype=tf.int32, name="padding_mask"),
    "mask_positions": keras.Input(shape=(None,), dtype=tf.int32, name="mask_positions"),
}

# Encoded token sequence
sequence = backbone(inputs)["sequence_output"]

# Predict an output word for each masked input token.
# We use the input token embedding to project from our encoded vectors to
# vocabulary logits, which has been shown to improve training efficiency.
outputs = mlm_head(sequence, mask_positions=inputs["mask_positions"])

# Define and compile our pretraining model.
pretraining_model = keras.Model(inputs, outputs)
pretraining_model.summary()
pretraining_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.AdamW(learning_rate=5e-4),
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
    jit_compile=True,
)

# Pretrain on IMDB dataset
pretraining_model.fit(
    pretrain_ds,
    validation_data=pretrain_val_ds,
    epochs=6,  # Increase to 6 for higher accuracy
)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ mask_positions            │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ padding_mask (InputLayer) │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ segment_ids (InputLayer)  │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ token_ids (InputLayer)    │ (None, None)           │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bert_backbone             │ [(None, 128), (None,   │      4,385,920 │ mask_positions[0][0],  │
│ (BertBackbone)            │ None, 128)]            │                │ padding_mask[0][0],    │
│                           │                        │                │ segment_ids[0][0],     │
│                           │                        │                │ token_ids[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ masked_lm_head            │ (None, None, 30522)    │      3,954,106 │ bert_backbone[0][1],   │
│ (MaskedLMHead)            │                        │                │ mask_positions[0][0]   │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,433,210 (16.91 MB)

 Trainable params: 4,433,210 (16.91 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/6


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:576: UserWarning: Gradients do not exist for variables ['kernel', 'bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


1563/1563 ━━━━━━━━━━━━━━━━━━━━ 160s 64ms/step - loss: 5.6248 - sparse_categorical_accuracy: 0.0607 - val_loss: 4.9752 - val_sparse_categorical_accuracy: 0.1169
Epoch 2/6
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 49s 31ms/step - loss: 4.9935 - sparse_categorical_accuracy: 0.1209 - val_loss: 4.8664 - val_sparse_categorical_accuracy: 0.1344
Epoch 3/6
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 48s 30ms/step - loss: 4.8602 - sparse_categorical_accuracy: 0.1413 - val_loss: 4.5435 - val_sparse_categorical_accuracy: 0.1969
Epoch 4/6
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 90s 35ms/step - loss: 4.5951 - sparse_categorical_accuracy: 0.1914 - val_loss: 4.2552 - val_sparse_categorical_accuracy: 0.2382
Epoch 5/6
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 81s 34ms/step - loss: 4.3458 - sparse_categorical_accuracy: 0.2255 - val_loss: 4.1010 - val_sparse_categorical_accuracy: 0.2537
Epoch 6/6
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 49s 31ms/step - loss: 4.1705 - sparse_categorical_accuracy: 0.2455 - val_loss: 3.7268 - val_sparse_categorical_accuracy: 0.

### Build and train your own transformer from scratch

#### Train custom vocabulary from IMDB data

In [13]:
vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
    imdb_train.map(lambda x, y: x),
    vocabulary_size=20_000,
    lowercase=True,
    strip_accents=True,
    reserved_tokens=["[PAD]", "[START]", "[END]", "[MASK]", "[UNK]"],
)
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=vocab,
    lowercase=True,
    strip_accents=True,
    oov_token="[UNK]",
)


#### Preprocess data with a custom tokenizer

In [14]:
packer = keras_nlp.layers.StartEndPacker(
    start_value=tokenizer.token_to_id("[START]"),
    end_value=tokenizer.token_to_id("[END]"),
    pad_value=tokenizer.token_to_id("[PAD]"),
    sequence_length=512,
)


def preprocess(x, y):
    token_ids = packer(tokenizer(x))
    return token_ids, y


imdb_preproc_train_ds = imdb_train.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)
imdb_preproc_val_ds = imdb_test.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE
).prefetch(tf.data.AUTOTUNE)

print(imdb_preproc_train_ds.unbatch().take(1).get_single_element())



(<tf.Tensor: shape=(512,), dtype=int32, numpy=
array([    1,   152,   410,   212,   115,  1020,   140,    43,  1648,
         104,   110,   106,   146,   145,  1776,   102,   148,    18,
         102,   100,    43,   155,   105,   384,   238,   270,  1120,
         236,   347,   250,    97,  7782,    18,    32,   101,    19,
          34,    32,   101,    19,    34,   105,   100,   128,  2601,
         103,    43,   270,    98,  1212,   191,   115,   105,    51,
       13316,    17,   130,   188,    13,    16,   111,   113,   119,
          99,   207,    43,   110,   105,  3097,   300,   102,    18,
           2,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,  

#### Design a tiny transformer

In [15]:
token_id_input = keras.Input(
    shape=(None,),
    dtype="int32",
    name="token_ids",
)
outputs = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=len(vocab),
    sequence_length=packer.sequence_length,
    embedding_dim=64,
)(token_id_input)
outputs = keras_nlp.layers.TransformerEncoder(
    num_heads=2,
    intermediate_dim=128,
    dropout=0.1,
)(outputs)
# Use "[START]" token to classify
outputs = keras.layers.Dense(2)(outputs[:, 0, :])
model = keras.Model(
    inputs=token_id_input,
    outputs=outputs,
)

model.summary()


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ token_ids (InputLayer)               │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding         │ (None, None, 64)            │       1,259,648 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_encoder_2                │ (None, None, 64)            │          33,472 │
│ (TransformerEncoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ get_item_6 (GetItem)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,293,250 (4.93 MB)

 Trainable params: 1,293,250 (4.93 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.AdamW(5e-5),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    jit_compile=True,
)
model.fit(
    imdb_preproc_train_ds,
    validation_data=imdb_preproc_val_ds,
    epochs=3,
)


Epoch 1/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 48s 21ms/step - loss: 0.6949 - sparse_categorical_accuracy: 0.5352 - val_loss: 0.5050 - val_sparse_categorical_accuracy: 0.7697
Epoch 2/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 21s 13ms/step - loss: 0.4403 - sparse_categorical_accuracy: 0.7940 - val_loss: 0.3318 - val_sparse_categorical_accuracy: 0.8627
Epoch 3/3
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 43s 14ms/step - loss: 0.3080 - sparse_categorical_accuracy: 0.8701 - val_loss: 0.3318 - val_sparse_categorical_accuracy: 0.8565
